# Paso 0: La Regla de Oro de McKinney (Preparación)

McKinney establece que cualquier función de ventana es inútil si la máquina no entiende el tiempo. Primero, cargamos, convertimos y aseguramos el orden.

In [1]:
import pandas as pd

# Leemos el archivo masivo y aplicamos las reglas de McKinney: Datatime y Sort

df_retail = (
    pd.read_csv('files_folder/retail_global_dataset.csv')
    .assign(fecha = lambda x: pd.to_datetime(x['fecha'])) # Convertimos a formato de tiempo real
    .sort_values('fecha')   # Ordenar las filas del DataFrame según la columna 'fecha' en orden ascendente.
    .reset_index(drop=True) # Se está restableciendo el índice del DataFrame df a un índice numérico secuencial que empieza en 0
)

print(f"Datos listos: {df_retail.shape[0]} filas ordenadas desde {df_retail['fecha'].min()} hasta {df_retail['fecha'].max()}")

Datos listos: 5000 filas ordenadas desde 2023-01-01 09:00:00 hasta 2024-12-31 00:00:00


Aquí se importa pandas y se construye el DataFrame df_retail encadenando operaciones: primero se lee el archivo CSV, luego se crea (o transforma) la columna fecha convirtiéndola a tipo datetime con pd.to_datetime() para poder trabajar correctamente con fechas, después se ordenan las filas de forma ascendente según esa columna para dejar los datos en orden cronológico y finalmente se restablece el índice para que quede secuencial desde 0 tras el ordenamiento; por último, se imprime un resumen indicando cuántas filas tiene el dataset y el rango temporal que abarcan los datos (fecha mínima y máxima).

# Ejemplo 1: .transform() - El Contexto (Effective Pandas)

La Teoría: Matt Harrison dice que .transform() sirve para que una fila "conozca" el resumen de su grupo sin colapsar la tabla (Feature Engineering).
El Caso Real: Queremos saber qué porcentaje representa una transacción específica dentro del total de ventas de su país.

In [2]:
df_peso = df_retail.assign(
   # 1. Calculamos el total de dinero generado por el país de esta transacción
    total_pais_usd = lambda x:x.groupby('pais')['ingreso_total'].transform('sum'),

    # 2. Dividimos la venta individual entre el total de su país para sacar el porcentaje
    peso_porcentual = lambda x:(x['ingreso_total']/x['total_pais_usd'])*100
)

print("---- PESO DE CADA VENTA EN SU PAIS----")
print(df_peso[['id_transaccion', 'pais', 'ingreso_total', 'total_pais_usd', 'peso_porcentual']].head())

---- PESO DE CADA VENTA EN SU PAIS----
  id_transaccion      pais  ingreso_total  total_pais_usd  peso_porcentual
0     TRX-000745    España        4217.37      1736686.67         0.242840
1     TRX-000531    España         914.76      1736686.67         0.052673
2     TRX-001595     Chile        3953.44      2683744.32         0.147311
3     TRX-001446    España        2298.45      1736686.67         0.132347
4     TRX-004218  Alemania        1900.80      2255873.10         0.084260


En este bloque se crea un nuevo DataFrame df_peso a partir de df_retail agregando dos columnas: primero, total_pais_usd, que calcula el total de ingresos por país usando groupby('pais') y transform('sum') para que cada fila tenga el total correspondiente a su país; luego, peso_porcentual, que obtiene el porcentaje que representa cada venta individual respecto al total de su país dividiendo ingreso_total entre total_pais_usd y multiplicando por 100; finalmente, se imprime un pequeño resumen mostrando el identificador de la transacción, el país, el ingreso individual, el total del país y el peso porcentual, permitiendo ver de manera clara la contribución de cada venta dentro de su mercado nacional

# Ejemplo 2: .cumsum() - La Ventana Expansiva (Python for Data Analysis)

La Teoría: McKinney explica que esto retiene el detalle de la fila pero va acumulando el historial.
El Caso Real: El Director Global quiere ver el "Year-to-Date" (YTD). Quiere ver cómo crece el dinero transacción por transacción, pero separado por Región.

In [3]:
# ytd es abrevación de 'Year-to-Date
df_ytd = df_retail.assign(
    # Agrupamos por región y empezamos a sumar como una bola de nieve
    ingreso_acumulado_region = lambda x:x.groupby('region')['ingreso_total'].cumsum(),
)

print("---- Crecimiento acumulado por región----")
# Filtramos solo EMEA para ver cómo su contador sube sin mezclarse con LATAM
print(df_ytd.query("region == 'EMEA'")[['fecha', 'region', 'ingreso_total', 'ingreso_acumulado_region']].head(8))

---- Crecimiento acumulado por región----
                 fecha region  ingreso_total  ingreso_acumulado_region
0  2023-01-01 09:00:00   EMEA        4217.37                   4217.37
1  2023-01-01 09:00:00   EMEA         914.76                   5132.13
3  2023-01-01 13:00:00   EMEA        2298.45                   7430.58
4  2023-01-01 16:00:00   EMEA        1900.80                   9331.38
10 2023-01-03 07:00:00   EMEA        3465.60                  12796.98
11 2023-01-03 12:00:00   EMEA       13396.34                  26193.32
17 2023-01-04 14:00:00   EMEA        6628.44                  32821.76
20 2023-01-05 09:00:00   EMEA         366.85                  33188.61


Calcula el ingreso acumulado Year-to-Date (YTD) por región dentro del DataFrame df_retail. Primero, con assign(), crea una nueva columna llamada ingreso_acumulado_region. Para calcularla, usa groupby('region') para separar los datos por cada región (por ejemplo, EMEA, LATAM, etc.) y luego aplica cumsum() sobre la columna ingreso_total, lo que va sumando progresivamente los ingresos dentro de cada grupo en orden de aparición, sin mezclar regiones entre sí (es decir, el acumulado de EMEA no incluye valores de LATAM). Después, imprime un encabezado descriptivo y utiliza query("region == 'EMEA'") para filtrar únicamente las filas de la región EMEA, mostrando las primeras 8 filas con head(8) y seleccionando solo las columnas relevantes (fecha, region, ingreso_total e ingreso_acumulado_region) para observar cómo el ingreso se va acumulando de manera incremental dentro de esa región.

# Ejemplo 3: .rolling() - El Suavizado de Ruido (Series de Tiempo)

La Teoría: McKinney ubica a .rolling() en el análisis de Series de Tiempo para eliminar el ruido estadístico.
El Caso Real: Tienes transacciones de $50 y transacciones de $5,000 en la misma hora. Eso crea una gráfica caótica. Vamos a calcular el Promedio Móvil de las últimas 50 transacciones para ver la verdadera tendencia de compra de la empresa.

In [8]:
df_tendencia = df_retail.assign(
    # La ventana toma las últimas 50 ventas (incluyendo la actual), las promedia y avanza
    promedio_movil_50 =  lambda x:x['ingreso_total'].rolling(window=50).mean()
)

print("---- Tendencia de ventas (Promedio Móvil 50 TRX)----")
# Mostramos apartir de la fila 48 para que veas dónde termina el NaN y empieza el cálculo
print(df_tendencia[['fecha', 'ingreso_total', 'promedio_movil_50']].iloc[49:53])

---- Tendencia de ventas (Promedio Móvil 50 TRX)----
                 fecha  ingreso_total  promedio_movil_50
49 2023-01-09 00:00:00        8089.08          3748.5542
50 2023-01-09 01:00:00        9645.46          3857.1160
51 2023-01-09 03:00:00        1284.70          3864.5148
52 2023-01-09 05:00:00         707.30          3799.5920


Este bloque de código crea un nuevo DataFrame llamado `df_tendencia` a partir de `df_retail`, añadiendo una columna llamada `promedio_movil_50` mediante `assign()`. Esa nueva columna se calcula con `rolling(window=50).mean()`, lo que significa que para cada fila se toma una **ventana móvil de 50 registros consecutivos** de la columna `ingreso_total` (incluyendo el actual), se calcula su promedio y luego la ventana “avanza” una fila, generando así una medida suavizada de la tendencia de ventas; las primeras 49 filas quedan como `NaN` porque no hay suficientes datos para completar las 50 observaciones requeridas. Luego, se imprime un encabezado descriptivo y se muestran únicamente las columnas `fecha`, `ingreso_total` y `promedio_movil_50`. Aquí es clave entender `iloc[48,53]`: `iloc` permite seleccionar datos **por posición numérica** (índices enteros), no por etiquetas; el primer número indica la fila y el segundo la columna. Sin embargo, esa sintaxis está intentando acceder a una sola celda específica (fila 48, columna 53), mientras que el comentario sugiere que se quiere mostrar un rango de filas (por ejemplo, de la 48 a la 53). Para eso, lo correcto sería usar `iloc[48:53]`, donde el operador `:` indica un **slice** (segmento) de filas por posición, permitiendo visualizar el punto exacto donde los valores `NaN` terminan y comienza a calcularse el promedio móvil.


# Solución 4: .shift() - La Máquina del Tiempo

La Teoría: Las comparaciones requieren que el "pasado" y el "presente" vivan en la misma fila.
El Caso Real: ¿Qué transacción generó más dinero que la inmediatamente anterior? Usamos .shift(1) para jalar el ingreso de la transacción previa a la fila actual y calcular la diferencia.

In [ ]:
df_diferencia = df_retail.assign(
    # Movemos toda la columna de ingresos un renglón hacia abajo
    ingreso_trx_anterior = lambda x:x['ingreso_total'].shift(1),

    # Restamos para ver si vendimos más o menos que en la transacción pasada
    crecimiento_vs_anterior = lambda x:x['ingreso_total']-x['ingreso_trx_anterior']
)

print("------ Comparación contra la transacción pasada ------")
print(df_diferencia[['id_transaccion', 'ingreso_total', 'ingreso_trx_anterior', 'crecimiento_vs_anterior']].head())